In [725]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
from time import sleep
import pandas as pd
import numpy as np

In [726]:
URL = "https://www.indeed.com/jobs?q=data+scientist+$20,000&l=New+York&start=11"

#conducting a request of the stated URL above:
page = requests.get(URL)
#specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.text, "html.parser")
#printing soup in a more structured tree format that makes for easier reading


In [495]:
#Retrieving Job Title
def extract_job_title_from_result(soup):
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)
extract_job_title_from_result(soup)

['Data Scientist',
 'Applied Machine Learning Scientist/Engineer - Unified Search Team',
 'Data Analyst / Data Scientist',
 'Presidential Associate Program Fall 2019',
 'Data Scientist Intern',
 'Data Scientist',
 'Data Warehouse Intern',
 'Data Scientist',
 'Data Scientist, Engineering',
 'Quantitative Data Analyst',
 'Data Scientist Intern',
 'Data Scientist',
 'AI Research Scientist',
 'Data Scientist',
 'Data Scientist']

In [496]:
#Company Info
def extract_company_from_result(soup): 
    companies = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                companies.append(span.text.strip())
    return(companies)
extract_company_from_result(soup)

['Fora Financial LLC',
 'Bloomberg',
 'Defined Clarity',
 'Estee Lauder',
 'IBM',
 'Net-Consultants',
 'Newsela',
 'Narrativ',
 'Google',
 'Flatiron Health',
 'The New York Times',
 'National Debt Relief',
 'Bloomberg',
 'Alliant Cooperative Data Solutions',
 'National Debt Relief']

In [497]:
def extract_location_from_result(soup): 
    locations = []
    spans = soup.findAll('span', attrs={'class': 'location'})
    for span in spans:
        locations.append(span.text)
    return(locations)
extract_location_from_result(soup)

['New York, NY',
 'New York, NY',
 'Armonk, NY 10504',
 'New York, NY',
 'New York, NY',
 'New York, NY',
 'New York, NY 10011 (Chelsea area)',
 'New York, NY',
 'New York, NY',
 'New York, NY']

In [498]:

def extract_salary_from_result(soup): 
    salaries = []
    for td in soup.find_all(name="td", attrs={"class":"snip"}):
            try:
                divs = td.find('div', attrs={'class': ''})
                spans = divs.find('span', {'class': 'no-wrap'})
                salaries.append(spans.text.strip())
            except:
                salaries.append("Nothing_found")
    return(salaries)
extract_salary_from_result(soup)

['Nothing_found',
 'Nothing_found',
 '$65 - $75 an hour',
 'Nothing_found',
 'Nothing_found',
 '$130,000 - $150,000 a year',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found',
 'Nothing_found']

In [499]:
def extract_summary_from_result(soup): 
    summaries = []
    spans = soup.findAll('span', attrs={'class': 'summary'})
    for span in spans:
        summaries.append(span.text.strip())
    return(summaries)
extract_summary_from_result(soup)

['A Day in the Life of a Data Scientist:. Identify new data sources/patterns that add significant lift to predictive modeling capabilities....',
 'Information and data move the financial markets. This data would be undiscoverable without our help. Bloomberg ingests more than 70,000 different news feeds,...',
 'We have a client that is looking for a data scientist to add to its team. Call Center or Customer care data experience a plus....',
 'Data Analytics PA. The PA will be given challenging projects that will focus on bridging the gaps between traditional marketers and data scientists....',
 'You will work alongside consultants, managers, and data scientists to enhance business performance through analytics and data science focused initiatives....',
 'This is an exciting opportunity if you are interested in predictive analytics, Big Data, data science and machine learning....',
 'Work with Data Analysts, Data Engineers, and Data Scientists to improve how data is modeled and organized

In [500]:
max_results_per_city = 100
city_set = ['New+York','Chicago','San+Francisco', 'Austin', 'Seattle', 'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Washington+DC', 'Boulder']
columns = ["city", "job_title", "company_name", "summary", "salary"]
sample_df = pd.DataFrame(columns = columns)

In [ ]:
for city in city_set:
    for start in range(0, max_results_per_city, 10):
        page = requests.get('http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=' + str(city) + '&start=' + str(start))
        time.sleep(1)  #ensuring at least 1 second between page grabs
        soup = BeautifulSoup(page.text, "lxml", from_encoding="utf-8")
        for div in soup.find_all(name="div", attrs={"class":"row"}): 
            #specifying row num for index of job posting in dataframe
            num = (len(sample_df) + 1) 
            #creating an empty list to hold the data for each posting
            job_post = [] 
            #append city name
            job_post.append(city) 
            #grabbing job title
            for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
                job_post.append(a["title"]) 
            #grabbing company name
            company = div.find_all(name="span", attrs={"class":"company"}) 
            if len(company) > 0: 
                for b in company:
                    job_post.append(b.text.strip()) 
            else: 
                sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
                for span in sec_try:
                    job_post.append(span.text) 
            #grabbing location name
#             spans = div.findAll('span', attrs={'class': 'location'})
#             for span in spans:
#                 job_post.append(span.text.strip())
            #grabbing summary text
            d = div.findAll('span', attrs={'class': 'summary'}) 
            for span in d:
                job_post.append(span.text.strip()) 
            #grabbing salary
            #td = div.findAll('td', attrs={'class': 'snip'}) 
            for td in div.find_all(name="td", attrs={"class":"snip"}):
                try:
                    divs = td.find('div', attrs={'class': ''})
                    spans = divs.find('span', {'class': 'no-wrap'})
                    job_post.append(spans.text.strip())
                    flag = 1
                except:
                    job_post.append("Nothing_Found")
#             try:
#                 divs = td.find('div', attrs={'class': ''}).find('span', {'class': 'no-wrap'}).string
#                 job_post.append(divs.strip())
#             except:
#                 job_post.append('Nothing_found')
            #appending list of job post info to dataframe at index num
            
            sample_df.loc[num] = job_post
            
    #saving sample_df as a local csv file — define your own local path to save contents 
    sample_df.to_csv("[filepath].csv", encoding='utf-8')

/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:146: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [707]:
#reading csv with index_col = 0, otherwise I get an additional
#unnamed column of separate index values
scrape_data = pd.read_csv("[filepath].csv", index_col=0)
scrape_data.shape
#dropping any duplicate rows:
scrape_data = scrape_data.drop_duplicates()
scrape_data.reset_index(drop=True, inplace=True)
scrape_data.shape

(1724, 5)

In [708]:
scrape_data["salary"].value_counts()

Nothing_Found                               1529
$110,000 - $130,000 a year                     6
$100,000 - $120,000 a year                     5
$60,000 - $80,000 a year                       4
$15 an hour                                    4
$45,000 - $55,000 a year                       3
$65 - $75 an hour                              3
$75,000 - $100,000 a year                      2
$100,000 - $130,000 a year                     2
$27.17 - $36.60 an hour                        2
$108,281 - $140,765 a year                     2
$50,000 a year                                 2
$3,267 - $4,275 a month                        2
$40,000 - $72,000 a year                       2
$120,000 - $150,000 a year                     2
$25,000 - $100,000 a year                      2
$92,000 - $120,000 a year                      2
$110,000 - $150,000 a year                     2
$106,000 - $135,000 a year (Indeed est.)       2
$67,968 - $126,062 a year                      2
$90,000 - $100,000 a

In [709]:
scrape_data.head(40)

,city,job_title,company_name,summary,salary
0,New+York,Data Scientist,National Debt Relief,O SAS/SQL/R and/or Data Files & Data manipulat...,Nothing_Found
1,New+York,Applied Machine Learning Scientist/Engineer - ...,Bloomberg,Information and data move the financial market...,Nothing_Found
2,New+York,Data Scientist (RD0057),Ascensia Diabetes Care,Experience in and understanding of data and in...,Nothing_Found
3,New+York,AI Research Scientist,Bloomberg,The infrastructure and algorithms we develop a...,Nothing_Found
4,New+York,Senior Big Data Engineer,Rockstar New York,"In collaboration with other data engineers, da...",Nothing_Found
5,New+York,Data Scientist (Entry-Level),First Derivatives plc,First Derivatives plc is currently recruiting ...,Nothing_Found
6,New+York,"HBO Data Scientist, Applied Analytics",HBO,"As an Applied Data Scientist within Analytics,...",Nothing_Found
7,New+York,Junior Data Scientist,Interactive Brokers,Interactive Brokers Group (IBG) is looking for...,Nothing_Found
8,New+York,Data Scientist,Rent the Runway,Mine the data ecosystem & find fruitful signal...,Nothing_Found
9,New+York,Data Integrity Analyst,Squarespace,"Partner with the Data Architect, responsible f...",Nothing_Found


In [710]:
#some targeted cleaning of salary information to make parsing easier #—> remove “\n”, “$”, and “,”
scrape_data["salary"] = scrape_data["salary"].str.replace("\n", "")
scrape_data["salary"] = scrape_data["salary"].str.replace(",", "")
scrape_data["salary"] = scrape_data["salary"].str.replace("$", "")

In [711]:
scrape_data["og_salary_period"] = np.nan
#if the salary contains information on time period, save that time
#period string in the og_salary_period column
scrape_data.loc[scrape_data["salary"].str.contains("year"), "og_salary_period"] = "year"
scrape_data.loc[scrape_data["salary"].str.contains("month"), "og_salary_period"] = "month"
scrape_data.loc[scrape_data["salary"].str.contains("week"), "og_salary_period"] = "week"
scrape_data.loc[scrape_data["salary"].str.contains("day"), "og_salary_period"] = "day"
scrape_data.loc[scrape_data["salary"].str.contains("hour"), "og_salary_period"] = "hour"

In [712]:
scrape_data.head(40)

,city,job_title,company_name,summary,salary,og_salary_period
0,New+York,Data Scientist,National Debt Relief,O SAS/SQL/R and/or Data Files & Data manipulat...,Nothing_Found,NaN
1,New+York,Applied Machine Learning Scientist/Engineer - ...,Bloomberg,Information and data move the financial market...,Nothing_Found,NaN
2,New+York,Data Scientist (RD0057),Ascensia Diabetes Care,Experience in and understanding of data and in...,Nothing_Found,NaN
3,New+York,AI Research Scientist,Bloomberg,The infrastructure and algorithms we develop a...,Nothing_Found,NaN
4,New+York,Senior Big Data Engineer,Rockstar New York,"In collaboration with other data engineers, da...",Nothing_Found,NaN
5,New+York,Data Scientist (Entry-Level),First Derivatives plc,First Derivatives plc is currently recruiting ...,Nothing_Found,NaN
6,New+York,"HBO Data Scientist, Applied Analytics",HBO,"As an Applied Data Scientist within Analytics,...",Nothing_Found,NaN
7,New+York,Junior Data Scientist,Interactive Brokers,Interactive Brokers Group (IBG) is looking for...,Nothing_Found,NaN
8,New+York,Data Scientist,Rent the Runway,Mine the data ecosystem & find fruitful signal...,Nothing_Found,NaN
9,New+York,Data Integrity Analyst,Squarespace,"Partner with the Data Architect, responsible f...",Nothing_Found,NaN


In [713]:
#filtering out the results with salary data != “Nothing_found”
salary_data = scrape_data[scrape_data["salary"] != "Nothing_Found"]
#removing all rows in salary data from scrape data, and converting #all “Nothing_found” values to NaN, so that float salary values can #be easily reintegrated later
scrape_data = scrape_data[~scrape_data.isin(salary_data)].dropna(how='all')
scrape_data["salary"].replace("Nothing_Found",np.nan, inplace=True)
scrape_data["salary"].astype('float')


0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
20     NaN
21     NaN
22     NaN
23     NaN
24     NaN
25     NaN
26     NaN
28     NaN
29     NaN
31     NaN
        ..
1693   NaN
1694   NaN
1695   NaN
1697   NaN
1698   NaN
1699   NaN
1700   NaN
1701   NaN
1702   NaN
1703   NaN
1704   NaN
1705   NaN
1706   NaN
1707   NaN
1708   NaN
1709   NaN
1710   NaN
1711   NaN
1712   NaN
1713   NaN
1714   NaN
1715   NaN
1716   NaN
1717   NaN
1718   NaN
1719   NaN
1720   NaN
1721   NaN
1722   NaN
1723   NaN
Name: salary, Length: 1529, dtype: float64

In [714]:
scrape_data.shape

(1529, 6)

In [715]:
salary_data.head(40)

,city,job_title,company_name,summary,salary,og_salary_period
27,New+York,Data Analyst / Data Scientist,Defined Clarity,We have a client that is looking for a data sc...,65 - 75 an hour,hour
30,New+York,Data Scientist,Net-Consultants,This is an exciting opportunity if you are int...,130000 - 150000 a year,year
42,New+York,Data Scientist,3Q Digital,"Data Wrangling, Feature Engineering, Data Visu...",80000 - 125000 a year,year
44,New+York,Sr. Data Scientist,Kaiser Tech Inc,Senior Data Scientist. 4.Many years of experie...,50 - 80 an hour,hour
45,New+York,Bike Share Data Analyst,DEPARTMENT OF TRANSPORTATION,Develop analyses of data based on economic stu...,52143 - 78997 a year,year
50,New+York,Agriculture Data Scientist,Harnham,Agriculture Data Scientist. Their top priority...,120000 - 150000 a year,year
58,New+York,Data Scientist/Quant (Hedge Fund),Averity,"Are you a quantitative, alpha-driven data scie...",150000 - 225000 a year,year
59,New+York,Startup Seeking Investment Research - Credit A...,"Quant Media, Inc.",Quant Media is looking for an analyst with lev...,60000 - 70000 a year,year
84,New+York,Research Scientist,"Research Foundation for Mental Hygiene, Inc.","Develop data collection forms, collect and rep...",66875 - 85511 a year,year
100,Chicago,FBI Intelligence Analyst,Federal Bureau of Investigation,"Media Analysis (includes media science, social...",35000 - 72000 a year,year


In [716]:
#separating out data based on whether contains hour, day, week, #month, year


year_salaries = salary_data.loc[salary_data["salary"].str.contains("year")]
month_salaries = salary_data.loc[salary_data["salary"].str.contains("month")]
week_salaries = salary_data.loc[salary_data["salary"].str.contains("week")]
day_salaries = salary_data.loc[salary_data["salary"].str.contains("day")]
hour_salaries = salary_data.loc[salary_data["salary"].str.contains("hour")]
# removing string values(“ a year”, “ a week”, etc. from salary dfs)
year_salaries["salary"] = year_salaries["salary"].str.replace(" a year", "")
month_salaries["salary"] = month_salaries["salary"].str.replace(" a month", "")
week_salaries["salary"] = week_salaries["salary"].str.replace(" a week", "")
day_salaries["salary"] = day_salaries["salary"].str.replace(" a day", "")
hour_salaries["salary"] = hour_salaries["salary"].str.replace(" an hour", "")


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [717]:
def split_sal(i):
    try:
        print(i)
        #splt = i.split(' - ',1)
        #splt = [int(s) for s in i.split() if s.isdigit() || if s.isfloat()]
        splt = []
        for t in i.split():
            try:
                splt.append(float(t))
            except ValueError:
                pass
        #print(splt)
#         print(splt[1])
        first = float(splt[0])
        #print(first)
        second = float(splt[1])
        #print(second)
        return (first + second)/2
    except:
        return float(i)

In [718]:
year_salaries["salary"] = year_salaries["salary"].apply(split_sal)
month_salaries["salary"] = month_salaries["salary"].apply(split_sal)
month_salaries["salary"] = month_salaries["salary"] * 12
week_salaries["salary"] = week_salaries["salary"].apply(split_sal)
week_salaries["salary"] = week_salaries["salary"] * 52
day_salaries["salary"] = day_salaries["salary"].apply(split_sal)
day_salaries["salary"] = day_salaries["salary"] * 260
hour_salaries["salary"] = hour_salaries["salary"].apply(split_sal)
hour_salaries["salary"] = hour_salaries["salary"] * 2080

130000 - 150000
80000 - 125000
52143 - 78997
120000 - 150000
150000 - 225000
60000 - 70000
66875 - 85511
35000 - 72000
60000 - 95000
110000 - 130000
110000 - 130000
145000 - 175000
90000 - 140000
133000 - 170000 (Indeed est.)
106000 - 135000 (Indeed est.)
106000 - 135000 (Indeed est.)
87000 - 111000 (Indeed est.)
116000 - 148000 (Indeed est.)
106000 - 136000 (Indeed est.)
100000 - 128000 (Indeed est.)
120000 - 170000
145000 - 175000
120000 - 150000
135000 - 153000
125000 - 175000
100000 - 140000
130000 - 160000
40000 - 60000
40000
43769 - 62529
130000 - 155000
100000 - 130000
110000 - 150000
100000
100000
92000 - 120000
66300 - 94600
66300 - 94600
92000 - 120000
100000 - 120000
45000 - 55000
58245 - 97071
100000 - 120000
100000 - 120000
100000 - 120000
34000 - 39000
60000 - 80000
50000
76238 - 89708
35000 - 55000
38000 - 43000
100000 - 130000
110000 - 130000
100000 - 108000
90000 - 100000
125000 - 145000
50000
108281 - 140765
60000 - 110000
91631 - 119121
108281 - 140765
90000 - 100000

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [719]:
#rejoining salary data into main scrape_data df
combined_salaries = pd.concat([year_salaries, month_salaries, week_salaries, day_salaries, hour_salaries], axis=0)
scrape_data = pd.concat([scrape_data, combined_salaries], axis=0)

In [720]:
scrape_data

,city,job_title,company_name,summary,salary,og_salary_period
0,New+York,Data Scientist,National Debt Relief,O SAS/SQL/R and/or Data Files & Data manipulat...,NaN,NaN
1,New+York,Applied Machine Learning Scientist/Engineer - ...,Bloomberg,Information and data move the financial market...,NaN,NaN
2,New+York,Data Scientist (RD0057),Ascensia Diabetes Care,Experience in and understanding of data and in...,NaN,NaN
3,New+York,AI Research Scientist,Bloomberg,The infrastructure and algorithms we develop a...,NaN,NaN
4,New+York,Senior Big Data Engineer,Rockstar New York,"In collaboration with other data engineers, da...",NaN,NaN
5,New+York,Data Scientist (Entry-Level),First Derivatives plc,First Derivatives plc is currently recruiting ...,NaN,NaN
6,New+York,"HBO Data Scientist, Applied Analytics",HBO,"As an Applied Data Scientist within Analytics,...",NaN,NaN
7,New+York,Junior Data Scientist,Interactive Brokers,Interactive Brokers Group (IBG) is looking for...,NaN,NaN
8,New+York,Data Scientist,Rent the Runway,Mine the data ecosystem & find fruitful signal...,NaN,NaN
9,New+York,Data Integrity Analyst,Squarespace,"Partner with the Data Architect, responsible f...",NaN,NaN


In [721]:
#targeted replace of “r&d” in job titles and summaries with #“research development”, as I want to remove “&” 
#from listings as part of general clean-up
scrape_data["job_title"] = scrape_data["job_title"].str.replace("R&D", "research development")
scrape_data["summary"] = scrape_data["summary"].str.replace("R&D", "research development")

In [722]:
#creating data character cleaning function, and applying to all #columns, also lowercasing all string data for ease of later nlp
def data_clean(df, column):
    cleaning_list = ["+", "$","/",",","?",".",";","-","@","!","&","%","^","*",")","(", "\n"]
    for item in cleaning_list:
        df[column] = df[column].str.replace(item, " ")
        df[column] = df[column].str.lower()
#can’t clean the salary column due to float values, and don’t need #to clean og_salary, so keeping out of the for loop
for column in scrape_data.columns[0:len(scrape_data.columns)-2]:
    data_clean(scrape_data, column)

In [723]:
scrape_data

,city,job_title,company_name,summary,salary,og_salary_period
0,new york,data scientist,national debt relief,o sas sql r and or data files data manipulat...,NaN,NaN
1,new york,applied machine learning scientist engineer ...,bloomberg,information and data move the financial market...,NaN,NaN
2,new york,data scientist rd0057,ascensia diabetes care,experience in and understanding of data and in...,NaN,NaN
3,new york,ai research scientist,bloomberg,the infrastructure and algorithms we develop a...,NaN,NaN
4,new york,senior big data engineer,rockstar new york,in collaboration with other data engineers da...,NaN,NaN
5,new york,data scientist entry level,first derivatives plc,first derivatives plc is currently recruiting ...,NaN,NaN
6,new york,hbo data scientist applied analytics,hbo,as an applied data scientist within analytics ...,NaN,NaN
7,new york,junior data scientist,interactive brokers,interactive brokers group ibg is looking for...,NaN,NaN
8,new york,data scientist,rent the runway,mine the data ecosystem find fruitful signal...,NaN,NaN
9,new york,data integrity analyst,squarespace,partner with the data architect responsible f...,NaN,NaN


In [724]:
scrape_data.to_csv("scraped_clean.csv")